In [7]:
import numpy as np
import multiprocessing as mp
import random
import re

In [2]:
def softprojc(vec, i, c = -1e-5):
    return(np.where(vec < c, c, vec))
def softproji(vec, i):
    return(np.where(vec < 0, (-1 / np.sqrt(i)), vec))

In [8]:
# mode 1 samples rows
def weightsample(data, mode):
    prob = np.linalg.norm(data, axis=mode)
    return(prob / sum(prob))

In [4]:
def als(data, k, niter, reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    for j in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])

        for i in np.arange(niter):            
            # sample random row or column
            row = np.random.randint(data.shape[0])
            col = np.random.randint(data.shape[1])
            
            # perform linear reg update 
            rfactor[:, col] = np.matmul(np.linalg.pinv(lfactor), data[:, col])
            lfactor[row, :] = np.matmul(data[row, :], np.matmul(rfactor.T, np.linalg.inv(np.matmul(rfactor, rfactor.T))))
            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[niter - 1]):
            finalerror = seqerror[niter - 1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    return(lbest, rbest, lowesterror)

In [91]:
def rk(data, k, niter, kacziter, reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    for l in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])
        
        # outer loop for number of iterations 
        for i in np.arange(niter):
            approx = np.matmul(lfactor, rfactor)
            
            # weighted sampling of row and column from data approx matrix
            row = np.random.choice(data.shape[0], p = weightsample(approx, 1))
            col = np.random.choice(data.shape[1], p = weightsample(approx, 0))
            
            # inner loop for number of RK iterations
            for j in np.arange(kacziter):
                # sample index for entry of data matrix
                kaczrow = np.random.choice(lfactor.shape[0], p = weightsample(lfactor, 1))
                kaczcol = np.random.choice(rfactor.shape[1], p = weightsample(rfactor, 0))

                # compute RK step
                lfactor[row, :] = lfactor[row, :] + (data[row, kaczcol] - np.matmul(lfactor[row, :], rfactor[:, kaczcol])) / (np.linalg.norm(rfactor[:, kaczcol])**2) * rfactor[:, kaczcol].T 
                rfactor[:, col] = rfactor[:, col] + (data[kaczrow, col] - np.matmul(lfactor[kaczrow, :], rfactor[:, col])) / (np.linalg.norm(lfactor[kaczrow, :])**2) * lfactor[kaczrow, :]
     
            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[niter - 1]):
            finalerror = seqerror[niter - 1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    return(lbest, rbest, lowesterror)

In [176]:
def brk(data, k, s, niter, kacziter, reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    for l in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])
        
        # outer loop for number of iterations 
        for i in np.arange(niter):
            approx = np.matmul(lfactor, rfactor)
            
            # weighted sampling of row and column from data matrix
            #row = np.random.choice(data.shape[0], p = weightsample(approx, 1))
            #col = np.random.choice(data.shape[1], p = weightsample(approx, 0))
            row = np.random.choice(data.shape[0])
            col = np.random.choice(data.shape[1])
            
            # inner loop for number of RK iterations
            for j in np.arange(kacziter):
                # sample indices until at least one nonzero row or col
                
                resample = True
                while(resample):
                    rowsum = 0
                    colsum = 0
                    kaczrow = np.random.choice(lfactor.shape[0], size = s, replace = False)
                    kaczcol = np.random.choice(rfactor.shape[1], size = s, replace = False)

                    for samplerow in kaczrow:
                        rowsum = rowsum + sum(lfactor[samplerow, :])
                    for samplecol in kaczcol:
                        colsum = colsum + sum(rfactor[:, samplerow])
                    if (rowsum > 0 and colsum > 0):
                        resample = False

                # compute BRK step
                #kaczrow = np.random.choice(lfactor.shape[0], size = s, replace = False)
                #kaczcol = np.random.choice(rfactor.shape[1], size = s, replace = False)
                print("before")
                print(lfactor[row, :])
                print(rfactor[:, col])
                lfactor[row, :] = lfactor[row, :] + np.matmul((data[row, kaczcol] - np.matmul(lfactor[row, :], rfactor[:, kaczcol])), np.linalg.pinv(rfactor[:, kaczcol]))
                rfactor[:, col] = rfactor[:, col] + np.matmul(np.linalg.pinv(lfactor[kaczrow, :]), (data[kaczrow, col] - np.matmul(lfactor[kaczrow, :], rfactor[:, col])))
                print("after")
                print(lfactor[row, :])
                print(rfactor[:, col])
                
            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[niter - 1]):
            finalerror = seqerror[niter - 1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    return(lbest, rbest, lowesterror)

In [154]:
def gs(data, k, niter, gsiter, reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    for l in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])
        
        # outer loop for number of iterations 
        for i in np.arange(niter):
            approx = np.matmul(lfactor, rfactor)
            
            # weighted sampling of row and column from data matrix
            #row = np.random.choice(data.shape[0], p = weightsample(approx, 1))
            #col = np.random.choice(data.shape[1], p = weightsample(approx, 0))
            row = np.random.choice(data.shape[0])
            col = np.random.choice(data.shape[1])
            
            # inner loop for number of RK iterations
            for j in np.arange(gsiter):
                # sample indices for entry of data matrix, dont want norms in rk step to be 0
                #gsrow = np.random.choice(rfactor.shape[0], replace = False, p = weightsample(rfactor, 1))
                #gscol = np.random.choice(lfactor.shape[1], replace = False, p = weightsample(lfactor, 0))
                gsrow = np.random.choice(rfactor.shape[0])
                gscol = np.random.choice(lfactor.shape[1])
            
                # compute GS step
                rfactor[:, col] = rfactor[:, col] + np.matmul(lfactor[:, gscol].T, (data[:, col] - np.matmul(lfactor, rfactor[:, col])), np.eye(k)[:, gscol])
                lfactor[row, :] = lfactor[row, :] + np.matmul((data[row, :] - np.matmul(lfactor[row, :], rfactor)), np.matmul(np.array([rfactor[gsrow, :]]).T, np.array([np.eye(k)[gsrow, :]])))
                
                print(lfactor)
                print(rfactor)
            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[niter - 1]):
            finalerror = seqerror[niter - 1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    return(lbest, rbest, lowesterror)

In [171]:
def bgs(data, k, s, niter, gsiter, reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    for l in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])
        
        # outer loop for number of iterations 
        for i in np.arange(niter):
            approx = np.matmul(lfactor, rfactor)
            
            # weighted sampling of row and column from data matrix
            #row = np.random.choice(data.shape[0], p = weightsample(approx, 1))
            #col = np.random.choice(data.shape[1], p = weightsample(approx, 0))
            
            row = np.random.choice(data.shape[0])
            col = np.random.choice(data.shape[1])
            
            
            # inner loop for number of RK iterations
            for j in np.arange(gsiter):
                # sample indices for entry of data matrix, dont want norms in rk step to be 0
                #gsrow = np.random.choice(rfactor.shape[0], size = s, replace = False, p = weightsample(rfactor, 1))
                #gscol = np.random.choice(lfactor.shape[1], size = s, replace = False, p = weightsample(lfactor, 0))
                
                if s == 1:
                    gsrow = np.random.choice(rfactor.shape[0])
                    gscol = np.random.choice(lfactor.shape[1])
            
                    # compute GS step
                    rfactor[:, col] = rfactor[:, col] + np.matmul(lfactor[:, gscol].T, (data[:, col] - np.matmul(lfactor, rfactor[:, col])), np.eye(k)[:, gscol])
                    lfactor[row, :] = lfactor[row, :] + np.matmul((data[row, :] - np.matmul(lfactor[row, :], rfactor)), np.matmul(np.array([rfactor[gsrow, :]]).T, np.array([np.eye(k)[gsrow, :]])))
                
                else:
                    resample = True
                    while(resample):
                        rowsum = 0
                        colsum = 0
                        gsrow = np.random.choice(lfactor.shape[0], size = s, replace = False)
                        gscol = np.random.choice(rfactor.shape[1], size = s, replace = False)

                        for samplerow in gsrow:
                            rowsum = rowsum + sum(lfactor[samplerow, :])
                        for samplecol in gscol:
                            colsum = colsum + sum(rfactor[:, samplerow])
                        if (rowsum > 0 and colsum > 0):
                            resample = False

                    # compute BRK step
                    lfactor[row, :] = lfactor[row, :] + np.matmul((data[row, :] - np.matmul(lfactor[row, :], rfactor)), np.matmul(np.linalg.pinv(rfactor[gsrow, :]), np.eye(k)[:, gscol].T))
                    rfactor[:, col] = rfactor[:, col] + np.matmul(np.matmul(np.eye(k)[:, gscol], np.linalg.pinv(lfactor[:, gscol])), (data[:, col] - np.matmul(lfactor, rfactor[:, col])))

            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[niter - 1]):
            finalerror = seqerror[niter - 1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    return(lbest, rbest, lowesterror)

In [7]:
def alstest(data, k, niter, reinit = 1):
    A, S, e = als(data, k = k, niter = niter, reinit = reinit)
    approx = np.matmul(A, S)
    return((np.linalg.norm(data - approx) / np.linalg.norm(data)))

In [8]:
def rktest(data, k, niter, kacziter, reinit = 1):
    A, S, error = rk(data, k = k, niter = niter, kacziter = kacziter, reinit = reinit)
    approx = np.matmul(A, S)
    return((np.linalg.norm(data - approx) / np.linalg.norm(data)))

In [9]:
def brktest(data, k, s, niter, kacziter, reinit = 1):
    A, S, error = brk(data, k = k, s = s,  niter = niter, kacziter = kacziter, reinit = reinit)
    approx = np.matmul(A, S)
    return((np.linalg.norm(data - approx) / np.linalg.norm(data)))

In [10]:
def listener(q, textfile):
    '''listens for messages on the q, writes to file. '''
    with open(textfile, 'w') as f:
        while 1:
            m = q.get()
            if m == 'kill':
                f.write('killed')
                break
            f.write(str(m) + ', ')
            f.flush()

In [11]:
def read(filename): 
    with open(filename, 'r') as f:
        l = f.read().split(',')
    return(l)

In [12]:
def alswrite(data, k, niter, q):
    A, S, e = als(data, k = k, niter = niter)
    approx = np.matmul(A, S)
    q.put((np.linalg.norm(data - approx) / np.linalg.norm(data)))

In [13]:
def rkwrite(data, k, niter, kacziter, q):
    A, S, error = rk(data, k = k, niter = niter, kacziter = kacziter)
    approx = np.matmul(A, S)
    q.put((np.linalg.norm(data - approx) / np.linalg.norm(data)))

In [14]:
def brkwrite(data, k, s, niter, kacziter, q):
    A, S, error = brk(data, k = k, s = s, niter = niter, kacziter = kacziter)
    approx = np.matmul(A, S)
    q.put((np.linalg.norm(data - approx) / np.linalg.norm(data)))

In [15]:
def alsmp(data, k, niter, filename, loop, cores = mp.cpu_count()):
    manager = mp.Manager()
    q = manager.Queue()    
    pool = mp.Pool(cores)

    #put listener to work first
    watcher = pool.apply_async(listener, (q, filename))

    #fire off workers
    jobs = []
    for i in range(loop):
        job = pool.apply_async(alswrite, (data, 4, 100, q))
        jobs.append(job)

    # collect results from the workers through the pool result queue
    for job in jobs: 
        job.get()

    #now we are done, kill the listener
    q.put('kill')
    pool.close()
    pool.join()

In [16]:
def rkmp(data, k, niter, kacziter, filename, loop, cores = mp.cpu_count()): 
    manager = mp.Manager()
    q = manager.Queue()    
    pool = mp.Pool(cores)

    #put listener to work first
    watcher = pool.apply_async(listener, (q, filename))

    #fire off workers
    jobs = []
    for i in range(loop):
        job = pool.apply_async(rkwrite, (data, k, niter, kacziter, q))
        jobs.append(job)

    # collect results from the workers through the pool result queue
    for job in jobs: 
        job.get()

    #now we are done, kill the listener
    q.put('kill')
    pool.close()
    pool.join()

In [17]:
def brkmp(data, k, s, niter, kacziter, filename, loop, cores = mp.cpu_count()): 
    manager = mp.Manager()
    q = manager.Queue()    
    pool = mp.Pool(cores)

    #put listener to work first
    watcher = pool.apply_async(listener, (q, filename))

    #fire off workers
    jobs = []
    for i in range(loop):
        job = pool.apply_async(brkwrite, (data, k, s, niter, kacziter, q))
        jobs.append(job)

    # collect results from the workers through the pool result queue
    for job in jobs: 
        job.get()

    #now we are done, kill the listener
    q.put('kill')
    pool.close()
    pool.join()

In [18]:
def extracterr(tag, errfiles): 
    r = re.compile(".*(" + tag + ").*")
    files = list(filter(r.match, errfiles))
    title = list()
    meanerr = list()
    stderr = list()
    for f in reversed(files):
        title.append("".join(re.findall('[0-9]+k*', f)[0]))
        meanerr.append(np.mean(np.asarray(read(f)[:-1]).astype(float)))
        stderr.append(np.std(np.asarray(read(f)[:-1]).astype(float)))
    return(title, meanerr, stderr)

In [95]:
def alsupdate(data, lf, rf, s, siter):
    row = np.random.randint(data.shape[0])
    col = np.random.randint(data.shape[1])
            
    # perform linear reg update 
    for i in np.arange(siter):
        rf[:, col] = np.matmul(np.linalg.pinv(lf), data[:, col])
        lf[row, :] = np.matmul(data[row, :], np.matmul(rf.T, np.linalg.inv(np.matmul(rf, rf.T))))
    return(lf, rf)

In [79]:
def brkupdate(data, lf, rf, s, siter):
    approx = np.matmul(lfactor, rfactor)
            
    # weighted sampling of row and column from data matrix
    row = np.random.choice(data.shape[0], p = weightsample(approx, 1))
    col = np.random.choice(data.shape[1], p = weightsample(approx, 0))

    # loop for number of RK iterations
    for j in np.arange(siter):
        # if brk, sample indices until at least one nonzero row or col
        if (s > 1):
            resample = True
            while(resample):
                rowsum = 0
                colsum = 0
                kaczrow = np.random.choice(lfactor.shape[0], size = s, replace = False)
                kaczcol = np.random.choice(rfactor.shape[1], size = s, replace = False)

                for samplerow in kaczrow:
                    rowsum = rowsum + sum(lfactor[samplerow, :])
                for samplecol in kaczcol:
                    colsum = colsum + sum(rfactor[:, samplerow])
                if (rowsum > 0 and colsum > 0):
                    resample = False
        else:
            # weighted sampling if rk
            kaczrow = np.random.choice(lfactor.shape[0], p = weightsample(lfactor, 1))
            kaczcol = np.random.choice(rfactor.shape[1], p = weightsample(rfactor, 0))

        # compute BRK step
        lfactor[row, :] = lfactor[row, :] + np.matmul((data[row, kaczcol] - np.matmul(lfactor[row, :], rfactor[:, kaczcol])), np.linalg.pinv(rfactor[:, kaczcol]))
        rfactor[:, col] = rfactor[:, col] + np.matmul(np.linalg.pinv(lfactor[kaczrow, :]), (data[kaczrow, col] - np.matmul(lfactor[kaczrow, :], rfactor[:, col])))
    return(lf, rf)

In [94]:
def mf(data, k, s = 1, niter = 100, siter = 1, solver = 'als', reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    if solver == "als":
        f = alsupdate
    if solver == "brk":
        f = brkupdate
    if solver == "bgs":
        f = bgsupdate
    
    for l in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])
        
        # outer loop for number of iterations 
        for i in np.arange(niter):          
            lfactor, rfactor = f(data, lfactor, rfactor, s, kacziter)
            
            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[niter - 1]):
            finalerror = seqerror[niter - 1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    return(lbest, rbest, lowesterror)

In [65]:
def rk2(data, k, niter, kacziter, reinit = 1):
    # set to negative one so we can guarantee an update for the first init
    finalerror = -1
    
    # need to compare final error to overall best and store the overall best
    seqerror = np.empty(niter)
    lowesterror = np.empty(1)
    
    # store overall best factor matrices
    lbest = np.random.rand(data.shape[0], k)
    rbest = np.random.rand(k, data.shape[1])
    
    for l in np.arange(reinit):
        # randomly initialize the factor matrices
        lfactor = np.random.rand(data.shape[0], k)
        rfactor = np.random.rand(k, data.shape[1])
        
        # outer loop for number of iterations 
        for i in np.arange(niter):          
            lfactor, rfactor = brkupdate(data, lfactor, rfactor, kacziter)
            
            # calculate error after update
            seqerror[i] = np.linalg.norm(data - np.matmul(lfactor, rfactor)) / np.linalg.norm(data)
        # update after first init
        if (finalerror == -1):
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
        # if not first, only update if final error is lower than overall best
        elif (finalerror > seqerror[niter - 1]):
            finalerror = seqerror[niter - 1]
            lowesterror = seqerror
            lbest = lfactor
            rbest = rfactor
    return(lbest, rbest, lowesterror)

In [172]:
bfactor200 = np.random.choice(4, size=(200,4), p=np.array([0.7, 0.1, 0.1, 0.1]))
bweight200 = np.random.randint(0, 2, size=(4, 200))
data = np.matmul(bfactor200, bweight200)

print(data)

A, S, error = bgs(b200, 4, 1, 10, 3)
#A, S, error = gs(b200, 4, 10, 3)
print(error)

[[0 0 2 ... 2 0 2]
 [0 0 0 ... 0 0 0]
 [1 1 1 ... 1 1 1]
 ...
 [0 0 1 ... 1 0 1]
 [0 0 0 ... 3 3 3]
 [0 0 0 ... 0 0 0]]
[7.99209186e+04 6.21152834e+19 1.77101606e+44 4.07473539e+92
            inf            nan            nan            nan
            nan            nan]


C:\Users\Edwin\anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: overflow encountered in matmul
C:\Users\Edwin\anaconda3\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in matmul
C:\Users\Edwin\anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in matmul


In [136]:
k = 4
bfactor200 = np.random.choice(4, size=(200,4), p=np.array([0.7, 0.1, 0.1, 0.1]))
bweight200 = np.random.randint(0, 2, size=(4, 200))
data = np.matmul(bfactor200, bweight200)

lfactor = np.random.rand(data.shape[0], k)
rfactor = np.random.rand(k, data.shape[1])

# outer loop for number of iterations 
approx = np.matmul(lfactor, rfactor)

# weighted sampling of row and column from data matrix
row = np.random.choice(data.shape[0], p = weightsample(approx, 1))
col = np.random.choice(data.shape[1], p = weightsample(approx, 0))

# inner loop for number of RK iterations
# sample indices for entry of data matrix, dont want norms in rk step to be 0
gsrow = np.random.choice(rfactor.shape[0], replace = False, p = weightsample(rfactor, 1))
gscol = np.random.choice(lfactor.shape[1], replace = False, p = weightsample(lfactor, 0))

# compute GS step
rfactor[:, col] = rfactor[:, col] + np.matmul(lfactor[:, gscol].T, (data[:, col] - np.matmul(lfactor, rfactor[:, col])), np.eye(k)[:, gscol])
lfactor[row, :] = lfactor[row, :] + np.matmul((data[row, :] - np.matmul(lfactor[row, :], rfactor)), np.matmul(np.array([rfactor[gsrow, :]]).T, np.array([np.eye(k)[gsrow, :]])))


In [177]:
bfactor200 = np.random.choice(4, size=(200,4), p=np.array([0.91, 0.03, 0.03, 0.03]))
bweight200 = np.random.randint(0, 2, size=(4, 200))
data = np.matmul(bfactor200, bweight200)

A, S, brkerror = brk(data, k = 4, s = 4, niter = 10, kacziter = 1)
print(brkerror)

before
[0.22999129 0.37539302 0.65228874 0.55027515]
[0.09093969 0.16407356 0.76860897 0.83172389]
after
[ 4.16333634e-16 -4.44089210e-16  1.11022302e-16 -1.33226763e-15]
[-8.88178420e-16 -1.11022302e-15  7.77156117e-16  2.99760217e-15]
before
[0.00150657 0.15400805 0.4045318  0.90520043]
[0.75184714 0.95703076 0.89729336 0.44198762]
after
[-1.05167611e-16  2.77555756e-16 -1.11022302e-15 -2.22044605e-16]
[-1.11022302e-16 -3.55271368e-15  8.21565038e-15 -9.99200722e-16]
before
[0.92645978 0.07061881 0.28615832 0.56141382]
[0.23215575 0.74590528 0.94308354 0.37372826]
after
[-1.40860068 -0.05564027  0.48570512  1.26250717]
[-1.44328993e-15  3.33066907e-16  7.77156117e-16 -1.11022302e-16]
before
[0.89148984 0.34824224 0.56883248 0.14177758]
[0.90151568 0.86285669 0.90431268 0.32316389]
after
[ 5.58237347  0.39268821 -0.9145095  -2.28785994]
[-9.99200722e-15 -2.33146835e-15  4.44089210e-15  1.02140518e-14]
before
[0.63336518 0.57955918 0.20514548 0.59665474]
[0.75288476 0.40962468 0.241161

In [133]:
np.eye(k)[gsrow, :]

array([1., 0., 0., 0.])